In [2]:
# coding=utf-8
"""
This module contains the tests for the exchanges classes
"""
from fastlane_bot import Bot, Config
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3,  CarbonV1, BancorV3
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.helpers.poolandtokens import PoolAndTokens
from fastlane_bot.helpers import TradeInstruction, TxReceiptHandler, TxRouteHandler, TxSubmitHandler, TxHelpers, TxHelper
from fastlane_bot.events.managers.manager import Manager
from fastlane_bot.events.interface import QueryInterface
from joblib import Parallel, delayed
import pytest
import math
import json
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *

#plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v3.3.1 (05/Oct/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.2 (2023-08-27)
UniswapV3 v0.0.2 (2023-08-27)
CarbonV1 v0.0.2 (2023-08-27)
BancorV3 v0.0.2 (2023-08-27)
Version = 3-b2.2 [requirements >= 3.0 is met]


# Multi Mode [NB039]

In [ ]:
C = cfg = Config.new(config=Config.CONFIG_MAINNET)
cfg.DEFAULT_MIN_PROFIT_GAS_TOKEN = 0.00001
assert (C.NETWORK == C.NETWORK_MAINNET)
assert (C.PROVIDER == C.PROVIDER_ALCHEMY)
setup_bot = CarbonBot(ConfigObj=C)
pools = None
with open('fastlane_bot/data/tests/latest_pool_data_testing.json') as f:
    pools = json.load(f)
pools = [pool for pool in pools]
pools[0]
static_pools = pools
state = pools
exchanges = list({ex['exchange_name'] for ex in state})
db = QueryInterface(state=state, ConfigObj=C, exchanges=exchanges)
setup_bot.db = db

static_pool_data_filename = "static_pool_data"

static_pool_data = pd.read_csv(f"fastlane_bot/data/{static_pool_data_filename}.csv", low_memory=False)
    
uniswap_v2_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v2_event_mappings.csv", low_memory=False)
        
tokens = pd.read_csv("fastlane_bot/data/tokens.csv", low_memory=False)
        
exchanges = "carbon_v1,bancor_v3,uniswap_v3,uniswap_v2,sushiswap_v2"

exchanges = exchanges.split(",")


alchemy_max_block_fetch = 20
static_pool_data["cid"] = [
        cfg.w3.keccak(text=f"{row['descr']}").hex()
        for index, row in static_pool_data.iterrows()
    ]
# Filter out pools that are not in the supported exchanges
static_pool_data = [
    row for index, row in static_pool_data.iterrows()
    if row["exchange_name"] in exchanges
]

static_pool_data = pd.DataFrame(static_pool_data)
static_pool_data['exchange_name'].unique()
# Initialize data fetch manager
mgr = Manager(
    web3=cfg.w3,
    w3_async=cfg.w3_async,
    cfg=cfg,
    pool_data=static_pool_data.to_dict(orient="records"),
    SUPPORTED_EXCHANGES=exchanges,
    alchemy_max_block_fetch=alchemy_max_block_fetch,
    uniswap_v2_event_mappings=uniswap_v2_event_mappings,
    tokens=tokens.to_dict(orient="records"),
)

# Add initial pools for each row in the static_pool_data
start_time = time.time()
Parallel(n_jobs=-1, backend="threading")(
    delayed(mgr.add_pool_to_exchange)(row) for row in mgr.pool_data
)
cfg.logger.info(f"Time taken to add initial pools: {time.time() - start_time}")

# check if any duplicate cid's exist in the pool data
mgr.deduplicate_pool_data()
cids = [pool["cid"] for pool in mgr.pool_data]
assert len(cids) == len(set(cids)), "duplicate cid's exist in the pool data"
def init_bot(mgr: Manager) -> CarbonBot:
    """
    Initializes the bot.

    Parameters
    ----------
    mgr : Manager
        The manager object.

    Returns
    -------
    CarbonBot
        The bot object.
    """
    mgr.cfg.logger.info("Initializing the bot...")
    bot = CarbonBot(ConfigObj=mgr.cfg)
    bot.db = db
    bot.db.mgr = mgr
    assert isinstance(
        bot.db, QueryInterface
    ), "QueryInterface not initialized correctly"
    return bot
bot = init_bot(mgr)
# add data cleanup steps from main.py
bot.db.remove_unmapped_uniswap_v2_pools()
bot.db.remove_zero_liquidity_pools()
bot.db.remove_unsupported_exchanges()
tokens = bot.db.get_tokens()
ADDRDEC = {t.address: (t.address, int(t.decimals)) for t in tokens if not math.isnan(t.decimals)}
flashloan_tokens = bot.setup_flashloan_tokens(None)
CCm = bot.setup_CCm(None)
pools = db.get_pool_data_with_tokens()

arb_mode = "multi"

2023-11-08 01:09:09,436 [fastlane:INFO] - 
2023-11-08 01:09:09,436 [fastlane:INFO] - **********************************************
2023-11-08 01:09:09,437 [fastlane:INFO] - The logging path is set to: logs/20231108-010909/bot.log
2023-11-08 01:09:09,437 [fastlane:INFO] - **********************************************
2023-11-08 01:09:09,437 [fastlane:INFO] - 
2023-11-08 01:09:11,252 [fastlane:INFO] - Retrieved 202 carbon pairs from contract
2023-11-08 01:09:12,519 [fastlane:INFO] - Time taken to add initial pools: 0.09212303161621094
2023-11-08 01:09:12,522 [fastlane:INFO] - Initializing the bot...
2023-11-08 01:09:12,716 [fastlane:INFO] - Removed 3242 unmapped uniswap_v2/sushi pools. 1897 uniswap_v2/sushi pools remaining
2023-11-08 01:09:12,716 [fastlane:INFO] - Unmapped uniswap_v2/sushi pools:
2023-11-08 01:09:12,903 [fastlane:INFO] - uniswap_v2: 3242
2023-11-08 01:09:12,904 [fastlane:INFO] - sushiswap_v2: 0
2023-11-08 01:09:12,906 [fastlane:INFO] - uniswap_v3: 636
2023-11-08 01:09:

## Test_MIN_PROFIT

In [ ]:
assert(cfg.DEFAULT_MIN_PROFIT_GAS_TOKEN <= 0.0001), f"[TestMultiMode], default_min_profit_gas_token must be <= 0.02 for this Notebook to run, currently set to {cfg.DEFAULT_MIN_PROFIT_GAS_TOKEN}"

## Test_validator_in_out

In [ ]:
arb_finder = bot._get_arb_finder("multi")
assert arb_finder.__name__ == "FindArbitrageMultiPairwise", f"[TestMultiMode] Expected arb_finder class name name = FindArbitrageMultiPairwise, found {arb_finder.__name__}"

## Test_validator_multi

In [ ]:
arb_finder = bot._get_arb_finder("multi")
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )
r = finder.find_arbitrage()

arb_opp = r[0]

validated = bot.validate_optimizer_trades(arb_opp=arb_opp, arb_mode="multi", arb_finder=finder)



assert arb_opp == validated


## Test_validator_single

In [ ]:
arb_mode="single"
arb_finder = bot._get_arb_finder(arb_mode)
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )

arb_opp = 111

validated = bot.validate_optimizer_trades(arb_opp=arb_opp, arb_mode=arb_mode, arb_finder=finder)


assert arb_opp == validated

## Test_validator_bancor_v3

In [ ]:
arb_mode="bancor_v3"

arb_finder = bot._get_arb_finder(arb_mode)
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )
r = finder.find_arbitrage()

arb_opp = None

for arb in r:
    has_carbon = False
    for trade in arb[2]:
        if '-' in trade['cid']:
            has_carbon = True
    
    if not has_carbon:
        arb_opp = arb
        break

validated = bot.validate_optimizer_trades(arb_opp=arb_opp, arb_mode=arb_mode, arb_finder=finder)

assert arb_opp != validated

2023-11-08 01:09:13,761 [fastlane:WARNING] - base_exchange must be bancor_v3 for single_triangle_bancor3, setting it to bancor_v3
2023-11-08 01:09:13,762 [fastlane:INFO] - flashloan_tokens for arb_mode=single_triangle_bancor3 will be overwritten. 
2023-11-08 01:09:13,763 [fastlane:INFO] - limiting flashloan_tokens to ['WETH-6Cc2', 'USDC-eB48', 'WBTC-C599', 'LINK-86CA', 'USDT-1ec7']
/Users/mikewcasale/Documents/GitHub/bancorprotocol/fastlane-bot/resources/NBTest/fastlane_bot/tools/optimizer/cpcarboptimizer.py:388: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc["PRICE"].fillna(1, inplace=True)
/Users/mikewcasale/Documents/GitHub/bancorprotocol/fastlane-bot/resources/NBTest/fastlane_bot/tools/optimizer/cpcarboptimizer.py:388: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

## Test_validator_multi_triangle

In [ ]:
arb_mode="multi_triangle"
arb_finder = bot._get_arb_finder(arb_mode)
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )

arb_opp = 1191

validated = bot.validate_optimizer_trades(arb_opp=arb_opp, arb_mode=arb_mode, arb_finder=finder)



assert arb_opp == validated